#File Operations

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/Studies/Intern/For_Colab/* .

In [ ]:
!nvidia-smi

Fri Aug 27 21:29:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

In [ ]:
!cp -r /content/drive/MyDrive/InternP1/models.zip .
!unzip -q models.zip
!rm models.zip

In [ ]:
!cp -r /content/drive/MyDrive/Studies/Intern/models .

In [ ]:
!cp -r /content/drive/MyDrive/Studies/InternP1/data.zip .
!unzip -d data/ data.zip
!rm data.zip

In [ ]:
#Remove folders
import shutil
shutil.rmtree('/content/data/test_predicts')
shutil.rmtree('/content/data/test')

In [ ]:
import os
os.remove("data/images/image_name...")

In [ ]:
!zip -r models.zip models
!cp models.zip /content/drive/MyDrive/Studies/InternP1/

!zip -r predicts.zip data/predicts
!cp predicts.zip /content/drive/MyDrive/Studies/InternP1/

#Video to Frame

In [ ]:
import cv2
# Opens the Video file
if not os.path.exists("data/"): 
  os.mkdir("data/")
if not os.path.exists("data/test"): 
  os.mkdir("data/test")

cap = cv2.VideoCapture('/content/drive/MyDrive/Studies/Intern/test/videos/test3_mahmutbey_short_cp.mp4')
i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('data/test/'+str(i)+'.jpg',frame)
    i+=1
 
cap.release()
cv2.destroyAllWindows()

#Frame to Video

In [ ]:
import cv2
import numpy as np
import os
from os.path import isfile, join
from natsort import natsorted
from tqdm import tqdm_notebook



pathIn= '/content/data/test_predicts/full_pred3_vid/'
pathOut = '/content/full_pred3_cp_vid.mp4'
fps = 24
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files = natsorted(files)
for i in tqdm_notebook(range(len(files))):
    filename=pathIn + files[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in tqdm_notebook(range(len(frame_array))):
    # writing to a image array
    out.write(frame_array[i])
out.release()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1536 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1536 [00:00<?, ?it/s]

In [ ]:
!cp /content/full_pred3_cp_vid.mp4 /content/drive/MyDrive/Studies/Intern/test/predicts

#Full Predict

In [ ]:
import os
import glob
import torch
import tqdm
import cv2
from tqdm import tqdm_notebook
from preprocess import tensorize_image
import numpy as np
from constant import *
from train import *
from natsort import natsorted

#### PARAMETERS #####
cuda = True
test = True
predict_name = "full_pred3_vid"
fs_model_name = "Unet_2.pt"
line_model_name = "SegNet.pt"
fs_model_path = os.path.join(MODELS_DIR, fs_model_name)
line_model_path = os.path.join(MODELS_DIR, line_model_name)
input_shape = input_shape
#####################

if test:
    if not os.path.exists(TEST_PREDICT_DIR): 
      os.mkdir(TEST_PREDICT_DIR)
    test_input_path_list = glob.glob(os.path.join(TEST_DIR, "*"))
    test_input_path_list = natsorted(test_input_path_list)
    predict_path = os.path.join(TEST_PREDICT_DIR, predict_name.split(".")[0])
else:
    if not os.path.exists(PREDICT_DIR): 
      os.mkdir(PREDICT_DIR)
    predict_path = os.path.join(PREDICT_DIR, predict_name.split(".")[0])

if not os.path.exists(predict_path): 
    os.mkdir(predict_path)

# LOAD MODEL
fs_model = torch.load(fs_model_path)
#Remember that you must call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference. 
#Failing to do this will yield inconsistent inference results.
fs_model.eval()

line_model = torch.load(line_model_path)
line_model.eval()

if cuda:
    fs_model = fs_model.cuda()
    line_model = line_model.cuda()



# PREDICT
def predict(fs_model, line_model, images):


    for image in tqdm_notebook(images):
        img = cv2.imread(image)
        batch_test = tensorize_image([image], input_shape, cuda)
        
        fs_output = fs_model(batch_test)
        line_output = line_model(batch_test)
        fs_out = torch.argmax(fs_output, axis=1)
        line_out = torch.argmax(line_output, axis=1)

        
        fs_out_cpu = fs_out.cpu()
        line_out_cpu = line_out.cpu()
        
        fs_outputs_list  = fs_out_cpu.detach().numpy()
        line_outputs_list  = line_out_cpu.detach().numpy()
        
        fs_mask = np.squeeze(fs_outputs_list, axis=0)
        line_mask = np.squeeze(line_outputs_list, axis=0)
       
        fs_mask_uint8 = fs_mask.astype('uint8')
        line_mask_uint8 = line_mask.astype('uint8')
        
        fs_mask_resize = cv2.resize(fs_mask_uint8, ((img.shape[1]), (img.shape[0])), interpolation = cv2.INTER_CUBIC)
        fs_line_resize = cv2.resize(line_mask_uint8, ((img.shape[1]), (img.shape[0])), interpolation = cv2.INTER_NEAREST)
        
        
        #img_resize = cv2.resize(img, input_shape)
        mask_ind = fs_mask_resize == 1
        mask_ind = fs_line_resize == 1
        #copy_img = img_resize.copy()
        copy_img = img.copy()
        
        img[fs_mask_resize==1, :] = (255, 0, 125)
        img[fs_line_resize==1, :] = (0, 0, 255)
        img[fs_line_resize==2, :] = (38, 255, 255)
        
        opac_image = (img/2 + copy_img/2).astype(np.uint8)
        cv2.imwrite(os.path.join(predict_path, image.split("/")[-1]), opac_image)
        #print("mask size from model: ", mask.shape),
        #print("resized mask size: ", mask_resize.shape)

if __name__ == "__main__":
    predict(fs_model, line_model, test_input_path_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1536 [00:00<?, ?it/s]

In [ ]:
!zip -r p1_test_full.zip data/test_predicts
!cp p1_test_full.zip /content/drive/MyDrive/Studies/Intern/Predicts